In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *



StatementMeta(, 01dd53a9-4052-46f8-8b9f-0a5797eea119, 3, Finished, Available)

In [2]:
%%sql
CREATE TABLE if not EXISTS lh_ecormece.Gold_DimProduct
(
    Product_ID long,
    Product_Category string,
    Product string,
    Created_TS timestamp,
    Modified_TS timestamp
)
using DELTA

StatementMeta(, 01dd53a9-4052-46f8-8b9f-0a5797eea119, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
max_date =spark.sql("""SELECT  coalesce(max(Modified_TS),"1900-01-01") from lh_ecormece.Gold_DimProduct""").first()[0]


StatementMeta(, 01dd53a9-4052-46f8-8b9f-0a5797eea119, 16, Finished, Available)

In [15]:
max_date

StatementMeta(, 01dd53a9-4052-46f8-8b9f-0a5797eea119, 17, Finished, Available)

'1900-01-01'

In [16]:
df_bronze= spark.sql("SELECT distinct  Product_Category,Product FROM lh_ecormece.bronze_sales where Modified_TS > '{}'".format(max_date))
display(df_bronze)
df_bronze.count()

StatementMeta(, 01dd53a9-4052-46f8-8b9f-0a5797eea119, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, b6d0a315-eb1a-4e36-bd94-a1df4bdb480c)

42

In [21]:
max_ID = spark.sql("select coalesce(max(Product_ID),0) from lh_ecormece.gold_dimproduct").first()[0]
max_ID

StatementMeta(, 01dd53a9-4052-46f8-8b9f-0a5797eea119, 23, Finished, Available)

0

In [22]:
df_final = df_bronze.withColumn("Product_ID",monotonically_increasing_id() + max_ID +1)
display(df_final)

StatementMeta(, 01dd53a9-4052-46f8-8b9f-0a5797eea119, 24, Finished, Available)

In [26]:
df_final.createOrReplaceTempView('ViewProduct')

StatementMeta(, 01dd53a9-4052-46f8-8b9f-0a5797eea119, 28, Finished, Available)

In [28]:
%%sql
merge into lh_ecormece.Gold_DimProduct as gp 
using ViewProduct as vp 
    on gp.Product_Category = vp.Product_Category and gp.Product = vp.Product
when MATCHED THEN
UPDATE set
    Modified_TS= current_timestamp() 
WHEN not MATCHED THEN
insert 
(
    gp.Product_ID,
    gp.Product_Category,
    gp.Product,
    gp.Created_TS,
    gp.Modified_TS
)
VALUES
(
    vp.Product_ID,
    vp.Product_Category,
    vp.Product,
    current_timestamp(),
    current_timestamp()
)

StatementMeta(, 01dd53a9-4052-46f8-8b9f-0a5797eea119, 30, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>